In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molsysmt as msm

_ColormakerRegistry()

# Elements selection

Elements selections is probably the most frequently task when we work with molecular systems. There are many circumstances under which we need to know list of elements acomplishing a certain condition. We probably need, for instance, to calculate de contact map between CA atoms from two chains, or to remove the solvent atoms or to know how many 'HIS' residues there are in a peptide. All these conditions can be expresed as a sentence that the query over elements needs to match. Each library or MD engine or molecular visualization software have each own syntaxis to write this sentence. You can see different examples in MDTraj, PyTraj, Amber, Pymol or VMD.

## MolSysMT selection syntaxis

MolSysMT has its own selection syntaxis based on the attributes of the elements as atoms, groups, molecules, etc. Lets load a molecular system to explain the logic behind this syntaxis:

In [7]:
file_path = msm.test_systems.files['1tcd.pdb']

In [8]:
molecular_system = msm.convert(file_path, to_form='molsysmt.MolSys')

A molecular system encoded as the native form 'MolSys' has a pandas DataFrame with the atoms breakdown:

In [ ]:
molecular_system.topology.elements

As you can see, the column names are the fundamental attributes of the molecular system elements:

In [ ]:
print(molecular_system.topology.elements.columns)

The syntaxis proposed by Pandas to perform queries in a pandas.DataFrame is the base of the MolSysMT selection procedure. The boolean syntaxis of Pandas includes the following words and symbols:

<center>

| Word | Symbol | Meaning |
|---|---|---|
| and | & | and |
| or | \| | or |
| not | ~ | not |
| in | | in |
|  | == | equal |
|  | != | not equal |
|  | < | less than |
|  | <= | less or equal than |
|  | > | greater than |
|  | >= | greater or equal than |

</center>

As such, the selection sentence can also include the reference to external lists. Lets see some simple examples.

### Simple atoms selection by their attributes or properties
The following are some examples where a list of atoms is obtained matching some selection criteria:

In [ ]:
# Atoms with name C
msm.select(molecular_system, 'atom.name == "C"')

In [ ]:
# Atoms with name CA or CB
msm.select(molecular_system, 'atom.name in ["CA","CB"]')

In [ ]:
# Atoms of type C or N
msm.select(molecular_system, 'atom.type==["C","N"]')

In [ ]:
# Heavy atoms
msm.select(molecular_system, 'not atom.type=="H"')

In [ ]:
# Atoms of type C not named CA
msm.select(molecular_system, 'atom.type=="C" and not atom.name=="CA"')

In [ ]:
# Atoms not named CA, CB or C
msm.select(molecular_system, 'atom.name!=["CA","CB","C"]')

In [ ]:
# Atoms with id number lower than 10
msm.select(molecular_system, 'atom.id<10')

In [ ]:
# Atoms with id number lower than 10 and higher or equal than 3
msm.select(molecular_system, 'atom.id<10 and atom.id>=3')

### Including other elements attributes

Atoms can be selected using attributes of other the other elements in the hierarchical organization of the molecular system: 'group', 'component', 'molecule', 'chain', 'entity' or 'bioassembly'. You can find further information of these elements in XXX. These are some examples of selection sentences including other criteria than atoms attributes:

In [ ]:
# Atoms belonging to molecules of type water.
msm.select(molecular_system, 'molecule.type=="water"')

In [ ]:
# Heavy atoms belonging to molecules of type protein.
msm.select(molecular_system, 'molecule.type=="protein" and atom.type!="H"')

In [ ]:
# Atoms belonging to residues named GLY, ALA or VAL in chain named A.
msm.select(molecular_system, 'group.name==["GLY","ALA","VAL"] and chain.name=="A"') 

### Including external variables

Pandas query method allows the use of external variables in the logical sentence. To include them, variables names have to be preceded by the character '@'. Lets illustrate its use with some examples:

In [ ]:
# Atoms in groups with indices 10, 11 or 12.
indices=[10,11,12]
msm.select(molecular_system, 'group.index==@indices')

In [ ]:
# Atoms named CA, C, O or N in groups with indices 10 to 29.
indices=list(range(10,30))
atoms=["CA", "C", "O", "N"]
msm.select(molecular_system, 'atom.name==@atoms & atom.index==@indices') 

### Including mask filters

Although including masks is not really necessary, `molsysmt.select()` has an optional input argument to do so:

In [ ]:
# Atoms named C with atom index in range 10 to 29
indices=list(range(10,30))
msm.select(molecular_system, 'atom.name=="C"', mask=indices)

The use of masks can always be avoid using the logical sentence:

In [ ]:
# Atoms named C with atom index in range 10 to 29
indices=list(range(10,30))
msm.select(molecular_system, 'atom.name=="C" and atom.index in @indices')

### Selection of other elements

The selection method of MolSysMT can also return other elements indices than atoms. As many methods in this library, `molsysmt.select()` has an input argument named `target` to select the elements nature of the output list of indices. Lets see some examples:

In [ ]:
# Groups with indices equal to 0, 100 or 200
indices=[0,100,200]
msm.select(molecular_system, 'group.index==@indices', target='group')

In [ ]:
# Groups with name "ALA"
msm.select(molecular_system, 'group.name=="ALA"', target='group')

In [ ]:
# Groups of atoms index 34, 44 or 64
msm.select(molecular_system, 'atom.index==[34,44,64]', target='group')

In [ ]:
# Groups belonging to chain named A and molecule of type anything but water
msm.select(molecular_system, 'chain.name=="A" and molecule.type!="water"', target='group')

In [ ]:
# Groups of molecules of type water
msm.select(molecular_system, 'molecule.type=="water"', target='group')

In [ ]:
# Molecules of type water
msm.select(molecular_system, 'molecule.type=="water"', target='molecule')

In [ ]:
# Chains with molecules of type water
msm.select(molecular_system, 'molecule.type=="water"', target='chain')

Finnally, notice that `mask` is always acting over the targeted elements:

In [ ]:
# Atoms with index from 0 to 4 and from 0 to 2
msm.select(molecular_system, 'atom.index in [0,1,2,3,4]', mask=[0,1,2], target='atom')

In [ ]:
# Groups with index from 0 to 4 and from 0 to 2
msm.select(molecular_system, 'group.index in [0,1,2,3,4]', mask=[0,1,2], target='group')

In [ ]:
# Molecules with index from 0 to 4 and from 0 to 2
msm.select(molecular_system, 'molecule.index in [0,1,2,3,4]', mask=[0,1,2], target='molecule')

## Syntaxis translation

MolSysMT is prepared to easily interact with other tools. The main goal of this library is providing with a set of pipes and joins to set up your workflows, keeping simple the integration of other tools. But different tools have different selection syntaxis. Learning how to use the selection syntaxis of MDTraj, ParmEd or NGLview is something very useful. Those are tools that we all use frequently in our labs. But it happens that we forget soon the rules of each tool. To keep a unique selection syntaxis in your projects, MolSysMT includes the input argument `to_syntaxis` in the method `molsysmt.select()`. Lets illustrate some examples:

In [ ]:
msm.select(molecular_system, selection='group.index==[3,4,5]', to_syntaxis='NGLview')

In [ ]:
msm.select(molecular_system, selection='group.index==[3,4,5]', to_syntaxis='MDTraj')

The output string can be obtained, if the selection is done over other targetted elements, as a sequence of groups or chains:

In [ ]:
msm.select(molecular_system, target='group', selection='group.index==[3,4,5]', to_syntaxis='NGLview')

In [ ]:
msm.select(molecular_system, target='group', selection='group.index==[3,4,5]', to_syntaxis='MDTraj')

### Output syntaxis supported

MolSysMT translates selection sentences from its own native syntaxis to NGLview, MDTraj, Pytraj, ParmEd and AMBER.

## Using your favourite selection syntaxis

To be implemented.